## Large Dataset


### Woking with package data.table in R

data.table is a mature package for fast data processing that presents an alternative to dplyr.

#### Read Data
 data.table supports rolling joins (which allow rows in one table to be selected based on proximity between shared variables (typically time) and non-equi joins (where join criteria can be inequalities rather than equal to).

In [ ]:
# open files
# using fread()
library(data.table)
n <- 10000
file <- "CollegeScorecard_Raw_Data/Scorecard_2009-2016.csv"
system.time({
  scores <- fread(file, nrows=n)
})

##    user  system elapsed 
##   4.616   0.120   4.737


system.time({
  scores2 <- read.csv(file, nrows=n)
})
##    user  system elapsed 
##  15.849   0.096  15.946

#### Data Manipulation
Here, NUMBRANCH is a column of the dataset and using data.table we can pull out certain rows by invoking the column name without having to write scores$NUMBRANCH

In [ ]:
# data manipulation
z <- scores[NUMBRANCH > 25]
nrow(z)
#253

scores[INSTNM == "University of North Carolina at Chapel Hill",1:20]

'''
We can also specify a column to be a key for the data.table. 
Specifying a key allows very fast subsetting based on the column you specify. Here, because the key is an integer, we wrap up the key in .(), 
otherwise it would interpret our request as a row number:
'''
setkey(scores, UNITID)
scores[.(199120),1:20] # 20 columns

setkey(scores, CITY)
scores["Chapel Hill",1:20]
##    UNITID    OPEID OPEID6                                      INSTNM
## 1: 199120 00297400   2974 University of North Carolina at Chapel Hill
## 2: 455141 04140700  41407                 Aveda Institute-Chapel Hill
##           CITY STABBR        ZIP ACCREDAGENCY INSTURL NPCURL SCH_DEG HCM2
## 1: Chapel Hill     NC      27599         NULL    NULL   NULL       3 NULL
## 2: Chapel Hill     NC 27514-7001         NULL    NULL   NULL       1 NULL
##    MAIN NUMBRANCH PREDDEG HIGHDEG CONTROL ST_FIPS REGION LOCALE
## 1:    1         1       3       4       1      37      5   NULL
## 2:    1         1       1       1       3      37      5   NULL

As you can see the key does not have to be unique (unlike row names in R which must be unique). Subsetting with a key column using data.table is much faster than subsetting via other methods. But the main takeaway should be that data.table is fast and if you have large datasets, you shouldn’t be using data.frame and base R functions for subsetting or grouping and summarization.

#### Functions inside the brackets

We can put functions inside of the square brackets.

We first convert TUITFTE to numeric, which gives a warning about NAs introduced in the coercion step:

In [ ]:
scores$TUITFTE <- as.numeric(scores$TUITFTE)
## Warning: NAs introduced by coercion

# use data.table
scores[,mean(TUITFTE,na.rm=TRUE)]

#grouping operations
scores[CONTROL==1,mean2(TUITFTE)]
scores[,mean2(TUITFTE),by=CONTROL]

#inour multiple functions
q25 <- function(x) quantile(x, .25, na.rm=TRUE)
q50 <- function(x) quantile(x, .50, na.rm=TRUE)
q75 <- function(x) quantile(x, .75, na.rm=TRUE)
scores[,.(median=q50(TUITFTE),q25=q25(TUITFTE),q75=q75(TUITFTE)),by=CONTROL]
##    CONTROL  median     q25      q75
## 1:       3  7621.5 4823.25 11503.25
## 2:       1  2549.0 1381.00  4635.00
## 3:       2 11631.5 7775.00 16654.25

### working with RSQLite
SQLite stores the entire database (definitions, tables, indices, and the data itself) as a single cross-platform file on a host machine. It implements this simple design by locking the entire database file during writing. SQLite read operations can be multitasked, though writes can only be performed sequentially.

If we wanted to try out the RQLite package without writing a file to disk we could have also used ":memory:" instead of writing a filename, which creates an in-memory database.

#### use package sqldf to write queries
library(sqldf)
sqldf('SELECT age, circumference FROM Orange WHERE Tree = 1 ORDER BY circumference ASC')


In [ ]:
#connect to myDB.sqlite

library(RSQLite)
library(DBI)
con <- dbConnect(SQLite(), "myDB.sqlite")
con
## <SQLiteConnection>
##   Path: /home/love/teach/statcomp/statcomp_src/large/myDB.sqlite
##   Extensions: TRUE

data(mtcars)
dbWriteTable(con, "cars", mtcars)
dbListTables(con)

#### queries

if there are more data than can fit in memory?

In [ ]:
rows <- dbGetQuery(con, "SELECT * FROM cars where ...")
head(rows)

#fetch chunks of data -- large datasets

#Here we formulate a query rs, and then fetch 10 rows at a time with dbFetch:
rs <- dbSendQuery(con, "SELECT * FROM cars")
d1 <- dbFetch(rs, n=10)
dbHasCompleted(rs)
#we can extract all remaining data by specifying -1:
d2 <- dbFetch(rs, n=-1)
dbHasCompleted(rs)

dbClearResult(rs)

dbDisconnect(con)
#Finally, we close the connection when we are finished working with the database:

### Working with RHDF5
The motivation for using an HDF5 data container is that, like SQLite we have a common format for representing a complex set of tables that can be shared simply be sharing a file, but unlike SQLite we are typically interested in reading in entire tables into memory, so that we can then analyze them. HDF5 is typically smaller on disk, as well as faster for writing or reading to or from disk, compared to SQLite.

An HDF5 data container is a standardized, highly-customizable data receptacle designed for portability. Unless your definition of ‘container’ is extremely broad, file systems are not commonly considered containers.

File systems aren’t portable: For example, you might be able to mount an NTFS file system on an AIX machine, but the integers or floating point numbers written on an Intel processor will turn out to be garbage when read on a IBM Power processor.

HDF5 achieves portability by separating its “cargo” (data) from its environment (file system, processor architecture, etc.) and by encoding it in a self-describing file format. The HDF5 library serves the dual purpose of being a parser/encoder of this format and an API for user-level objects (datasets, groups, attributes, etc.).

…

The data stored in HDF5 datasets is shaped and it is typed. Datasets have (logically) the shape of multi-dimensional rectilinear arrays. All elements in a given dataset are of the same type, and HDF5 has one of the most extensive type systems and one that is user-extendable.

#### The rhdf5 package

In [ ]:
install.packages("BiocManager") # can be skipped after 1st time
BiocManager::install("rhdf5")
library(rhdf5)

'''
Typically, we may already have an HDF5 data container that we want to work with, 
but as in the SQLite lecture note, we will show how to create a new one first.
'''
h5file <- "myDB.h5"
h5createFile(h5file)

x <- matrix(rnorm(1e4),nrow=100)
h5write(x, h5file, "A/x")
y <- matrix(letters, nrow=13)
h5write(y, h5file,"A/C/y")
df <- data.frame(a=1L:5L,
                 b=seq(0,1,length.out=5),
                 c=letters[1:5],
                 stringsAsFactors=FALSE)
h5write(df, h5file, "B/df")
h5ls(h5file)
##   group name       otype   dclass       dim
## 0     /    A   H5I_GROUP                   
## 1    /A    C   H5I_GROUP                   
## 2  /A/C    y H5I_DATASET   STRING    13 x 2
## 3    /A    x H5I_DATASET    FLOAT 100 x 100
## 4     /    B   H5I_GROUP                   
## 5    /B   df H5I_DATASET COMPOUND         5

#### Reading objects
We can read out these objects using `h5read`. Note that the column names of the data.frame have been preserved:

In [ ]:
xx <- h5read(h5file, "A/x")
xx[1:3,1:3]
##            [,1]       [,2]       [,3]
## [1,] -2.9180159 -0.3099286  0.5671834
## [2,]  1.2320955 -1.5603322 -0.7619277
## [3,] -0.3517632  0.2978257  0.9193802

df2 <- h5read(h5file, "B/df")
head(df2)
##   a    b c
## 1 1 0.00 a
## 2 2 0.25 b
## 3 3 0.50 c
## 4 4 0.75 d
## 5 5 1.00 e

### Sparse Data
Let’s dive right into representing sparse matrices. Here we have a large-ish matrix wherein the non-zero elements make up only ~5% of the total:

In [ ]:
m <- matrix(rbinom(1e6, 1, .05), ncol=1e3) # rbinom function (n,size,prob) n-number of observations. If length(n) > 1, the length is taken to be the number required
m[1:5,1:5]
##      [,1] [,2] [,3] [,4] [,5]
## [1,]    0    0    0    0    0
## [2,]    0    0    0    0    0
## [3,]    0    0    0    0    0
## [4,]    0    0    0    0    0
## [5,]    1    0    0    0    0

# Getting the number from 1 to 9
x <- rep(1:9)
x
  
# Calling the dim() function to
# Set dimension of 3 * 3
dim(x) <- c(3, 3)

#prod() function in R Language is used to return the multiplication results of all the values present in its arguments.
#> prod(c(1,2,4,NA,5),2:3,na.rm=TRUE)- 240， 1*2*4*5*2*3=240 
prod(dim(m))
## [1] 1e+06
print(object.size(m), units="Mb") #-- This matrix takes up about 4 Mb in memory

#create sparse matrix
library(Matrix)
mm <- Matrix(m, sparse=TRUE)
mm[1:5,1:5]
## 5 x 5 sparse Matrix of class "dgCMatrix"
##               
## [1,] . . . . .
## [2,] . . . . .
## [3,] . . . . .
## [4,] . . . . .
## [5,] 1 . . . .
#The sparse version takes up less than 1/6 of the space of the dense version.

#### how to create sparse matrix


In [ ]:
s <- sparseMatrix(i=c(1,3,5),j=c(1,2,3),x=c(4,5,6),dims=list(6,4))
## 6 x 4 sparse Matrix of class "dgCMatrix"
##             
## [1,] 4 . . .
## [2,] . . . .
## [3,] . 5 . .
## [4,] . . . .
## [5,] . . 6 .
## [6,] . . . .

We can do many operations to sparse matrices using specialized functions which are different than the ones defined for regular matrices. These are described in ?dgCMatrix-class, but some of the important ones are %*%, crossprod, tcrossprod, solve, qr, lu. Using these operations will preserve the sparsity of the object (so keeping us under our memory budger), and will perform much faster than coercion to dense would, if the matrices have a high degree of sparsity.

Note that some operations destroy the sparsity, such as adding 1, and therefore must be avoided (in the case where the dense matrix would not fit in memory):

In [ ]:
s[1:10,1:10] + 1
s[1:10,1:10] * 2
image(s[1:100,1:100])

#### use of sparse matrices in glmnet

A common use case of sparse matrices is in prediction of a target, let’s call y, using a high-dimensional, sparse matrix of features x. We are often in situation that there are more features in x than there are observations (rows of x and length of y). In this case it may make sense to first try linear modeling of y on x, and to use some combination of L1 and L2 regularization to stabilize the regression. The glmnet package allows one to fit elastic net models for such a problem, where the x matrix can be sparse, and it builds off of the sparse matrices defined in the Matrix package. Read over the help file for the main function:

--overfitting

In [ ]:
library(glmnet)

n <- 1e3
nn <- 1e5
x <- sparseMatrix(i=sample(n,nn,TRUE),
                  j=sample(n,nn,TRUE),
                  dims=list(n,n))
beta <- rep(c(1,0),c(50,950))
y <- x %*% beta + rnorm(n,0,.25)
fit <- glmnet(x, y, family="gaussian", alpha=1)
plot(fit)